In [3]:
import re
from typing import Dict, Iterable, List

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
# import plotly
import pymorphy2
from IPython.display import display
from scipy.stats import pearsonr, spearmanr
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from tqdm import tqdm_notebook

plt.style.use("ggplot")
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sofi_flin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sofi_flin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
SEED = 42
np.random.seed(SEED)

## *Зашружаем данные о песнях и их исполнителях из разных датасетов и сохраняю в общий датасет строчки с именем исполнителя и названием трека при этом исключаем лишние знаки.*

In [11]:
df1 = pd.read_csv('spotify_songs.csv')

df1.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [12]:
df1.shape

(32833, 23)

In [13]:
df2 = pd.DataFrame()
df2['sentence'] = df1['track_artist']+ ' ' +df1['track_name']

In [14]:
df2

,sentence
0,Ed Sheeran I Don't Care (with Justin Bieber) -...
1,Maroon 5 Memories - Dillon Francis Remix
2,Zara Larsson All the Time - Don Diablo Remix
3,The Chainsmokers Call You Mine - Keanu Silva R...
4,Lewis Capaldi Someone You Loved - Future Human...
...,...
32828,Lush & Simon City Of Lights - Official Radio Edit
32829,Tegan and Sara Closer - Sultan & Ned Shepard R...
32830,Starkillers Sweet Surrender - Radio Edit
32831,Mat Zo Only For You - Maor Levi Remix


In [21]:
def remove_numbers_punctuation_and_hyphens(text):
    # Удаление чисел, знаков препинания и тире из строки
    cleaned_text = re.sub(r'[-\d.,?!:;()\'\"<>{}[\]\|/@#$%^&*_+=~`]', '', text)
    return cleaned_text

In [15]:
df3 = pd.read_csv('spotify_millsongdata.csv')

df3.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [18]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [16]:
df4 = pd.DataFrame()
df4['sentence'] = df3['artist'] + ' ' +df3['song']
df4

,sentence
0,ABBA Ahe's My Kind Of Girl
1,"ABBA Andante, Andante"
2,ABBA As Good As New
3,ABBA Bang
4,ABBA Bang-A-Boomerang
...,...
57645,Ziggy Marley Good Old Days
57646,Ziggy Marley Hand To Mouth
57647,Zwan Come With Me
57648,Zwan Desire


In [23]:
substring = 'love me all a sudden'

In [27]:
result = df4['sentence'].str.contains(substring)

# Вывод результатов
print(result)

0        False
1        False
2        False
3        False
4        False
         ...  
57645    False
57646    False
57647    False
57648    False
57649    False
Name: sentence, Length: 57650, dtype: bool


In [28]:
true_count = result.sum()

# Вывод результата
print("Количество значений True:", true_count)

Количество значений True: 0


In [22]:
def pd_remove_numbers_punctuation_and_hyphens(text):
    # Удаление чисел, знаков препинания и тире из строки
    cleaned_text = re.sub(r'[-\d.,?!:;()\'\"<>{}[\]\|/@#$%^&*_+=~`]', '', str(text))
    return cleaned_text

# Применение функции к столбцу 'sentence' DataFrame
df22 = pd.DataFrame()
df22['sentence'] = df2['sentence'].apply(pd_remove_numbers_punctuation_and_hyphens)

# Замена всех символов '\xa0' на пробел
df22['sentence'] = df22['sentence'].str.replace('\xa0', ' ')

In [23]:
df22

,sentence
0,Ed Sheeran I Dont Care with Justin Bieber Lou...
1,Maroon Memories Dillon Francis Remix
2,Zara Larsson All the Time Don Diablo Remix
3,The Chainsmokers Call You Mine Keanu Silva Remix
4,Lewis Capaldi Someone You Loved Future Humans...
...,...
32828,Lush Simon City Of Lights Official Radio Edit
32829,Tegan and Sara Closer Sultan Ned Shepard Remix
32830,Starkillers Sweet Surrender Radio Edit
32831,Mat Zo Only For You Maor Levi Remix


In [24]:
df22 = df22.dropna(subset=['sentence'])
df22

,sentence
0,Ed Sheeran I Dont Care with Justin Bieber Lou...
1,Maroon Memories Dillon Francis Remix
2,Zara Larsson All the Time Don Diablo Remix
3,The Chainsmokers Call You Mine Keanu Silva Remix
4,Lewis Capaldi Someone You Loved Future Humans...
...,...
32828,Lush Simon City Of Lights Official Radio Edit
32829,Tegan and Sara Closer Sultan Ned Shepard Remix
32830,Starkillers Sweet Surrender Radio Edit
32831,Mat Zo Only For You Maor Levi Remix


In [25]:
df22 = df22.drop(df22[df22['sentence'].str.contains('nan', na=False)].index)
df22

,sentence
0,Ed Sheeran I Dont Care with Justin Bieber Lou...
1,Maroon Memories Dillon Francis Remix
2,Zara Larsson All the Time Don Diablo Remix
3,The Chainsmokers Call You Mine Keanu Silva Remix
4,Lewis Capaldi Someone You Loved Future Humans...
...,...
32828,Lush Simon City Of Lights Official Radio Edit
32829,Tegan and Sara Closer Sultan Ned Shepard Remix
32830,Starkillers Sweet Surrender Radio Edit
32831,Mat Zo Only For You Maor Levi Remix


In [26]:
def pd_remove_numbers_punctuation_and_hyphens(text):
    # Удаление чисел, знаков препинания и тире из строки
    cleaned_text = re.sub(r'[-\d.,?!:;()\'\"<>{}[\]\|/@#$%^&*_+=~`]', '', str(text))
    return cleaned_text

# Применение функции к столбцу 'sentence' DataFrame
df44 = pd.DataFrame()
df44['sentence'] = df4['sentence'].apply(pd_remove_numbers_punctuation_and_hyphens)

# Замена всех символов '\xa0' на пробел
df44['sentence'] = df44['sentence'].str.replace('\xa0', ' ')

In [27]:
df44

,sentence
0,ABBA Ahes My Kind Of Girl
1,ABBA Andante Andante
2,ABBA As Good As New
3,ABBA Bang
4,ABBA BangABoomerang
...,...
57645,Ziggy Marley Good Old Days
57646,Ziggy Marley Hand To Mouth
57647,Zwan Come With Me
57648,Zwan Desire


In [28]:
df44 = df44.dropna(subset=['sentence'])
df44

,sentence
0,ABBA Ahes My Kind Of Girl
1,ABBA Andante Andante
2,ABBA As Good As New
3,ABBA Bang
4,ABBA BangABoomerang
...,...
57645,Ziggy Marley Good Old Days
57646,Ziggy Marley Hand To Mouth
57647,Zwan Come With Me
57648,Zwan Desire


In [29]:
df44 = df44.drop(df44[df44['sentence'].str.contains('nan', na=False)].index)
df44

,sentence
0,ABBA Ahes My Kind Of Girl
1,ABBA Andante Andante
2,ABBA As Good As New
3,ABBA Bang
4,ABBA BangABoomerang
...,...
57645,Ziggy Marley Good Old Days
57646,Ziggy Marley Hand To Mouth
57647,Zwan Come With Me
57648,Zwan Desire


In [29]:
def split_sentences(text):
    # Разбиение текста на предложения по символам ".", "!", "?", или "\n"
    sentences = re.split(r'[.!?]+|\n', text)
    
    # Удаление чисел и знаков препинания из каждого предложения
    cleaned_sentences = []
    for sentence in sentences:
        # Удаление чисел из предложения
        sentence = re.sub(r'\d+', '', sentence)
        # Удаление знаков препинания из предложения
        sentence = re.sub(r'[^\w\s]', '', sentence)
        cleaned_sentences.append(sentence.strip())
    
    # Удаление пустых строк из списка предложений
    cleaned_sentences = [sentence for sentence in cleaned_sentences if sentence]
    
    return cleaned_sentences

In [30]:
df = pd.DataFrame()
df = pd.concat([df, df22], ignore_index=True)
df.shape

In [32]:
df = pd.concat([df, df44], ignore_index=True)
df.shape

(90388, 1)

In [34]:
corpus = df.sentence.values

In [35]:
df

,sentence
0,Ed Sheeran I Dont Care with Justin Bieber Lou...
1,Maroon Memories Dillon Francis Remix
2,Zara Larsson All the Time Don Diablo Remix
3,The Chainsmokers Call You Mine Keanu Silva Remix
4,Lewis Capaldi Someone You Loved Future Humans...
...,...
90383,Ziggy Marley Good Old Days
90384,Ziggy Marley Hand To Mouth
90385,Zwan Come With Me
90386,Zwan Desire


In [36]:
df = df.dropna(subset=['sentence'])
df

,sentence
0,Ed Sheeran I Dont Care with Justin Bieber Lou...
1,Maroon Memories Dillon Francis Remix
2,Zara Larsson All the Time Don Diablo Remix
3,The Chainsmokers Call You Mine Keanu Silva Remix
4,Lewis Capaldi Someone You Loved Future Humans...
...,...
90383,Ziggy Marley Good Old Days
90384,Ziggy Marley Hand To Mouth
90385,Zwan Come With Me
90386,Zwan Desire


### *Предобработка данных*

In [37]:
import re
from typing import List, Iterable
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from tqdm import tqdm_notebook

# Загрузка стоп-слов и токенизатора NLTK для английского языка
nltk.download('stopwords')
nltk.download('punkt')

# Инициализация стеммера Snowball для английского языка
snowball_stemmer = SnowballStemmer('english')

# Загрузка списка стоп-слов для английского языка
stopword_set = set(stopwords.words("english"))

def tokenize(text: str) -> List[str]:
    # Токенизация текста с использованием word_tokenize() из NLTK
    return word_tokenize(text.lower())

def lemmatize(token: str) -> str:
    # Применение стеммера Snowball для лемматизации токенов
    return snowball_stemmer.stem(token)

def prepare_sentence_dataset(documents: Iterable[str]) -> List[List[str]]:
    tokenized_sentences = []
    for document in tqdm_notebook(documents):
        for sentence in nltk.sent_tokenize(document):
            # Токенизация и лемматизация токенов в предложении
            lemmatized_tokens = [lemmatize(token) for token in tokenize(sentence)]
            # Удаление стоп-слов
            tokenized_sentences.append(
                [token for token in lemmatized_tokens if token not in stopword_set]
            )
    return tokenized_sentences

# Подготовка набора предложений для анализа
sentence_dataset = prepare_sentence_dataset(corpus)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sofi_flin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sofi_flin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_71023/1610590425.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for document in tqdm_notebook(documents):


  0%|          | 0/90388 [00:00<?, ?it/s]

In [38]:
len(sentence_dataset)

90388

In [39]:
corpus[4]

'Lewis Capaldi Someone You Loved  Future Humans Remix'

In [40]:
sentence_dataset[4]

['lewi', 'capaldi', 'someon', 'love', 'futur', 'human', 'remix']

In [41]:
sentences = [' '.join(words) for words in sentence_dataset]

In [42]:
sentences

['ed sheeran dont care justin bieber loud luxuri remix',
 'maroon memori dillon franci remix',
 'zara larsson time diablo remix',
 'chainsmok call mine keanu silva remix',
 'lewi capaldi someon love futur human remix',
 'ed sheeran beauti peopl feat khalid jack win remix',
 'kati perri never realli rhab remix',
 'sam feldt post malon feat rani gattüso remix',
 'avicii tough love tiësto remix radio edit',
 'shawn mend cant gryffin remix',
 'ed sheeran cross feat chanc rapper pnb rock remix',
 'elli gould hate rhab remix',
 'loud luxuri bodi',
 'avicii sos laidback luke tribut remix radio edit',
 'martin garrix summer day feat macklemor patrick stump fall boy tiësto remix',
 'ed sheeran south border feat camila cabello cardi b andi jarvi remix',
 'aj mitchel friend eden princ remix',
 'david guetta say name feat bebe rexha j balvin luca steve remix',
 'sam smith danc stranger normani cheat code remix',
 'steve aoki let sondr remix',
 'avicii heaven david guetta morten remix',
 'rhab arou

In [43]:
# Имя файла для записи
filename = "sentences_4.txt"

# Запись предложений в файл
with open(filename, "w", encoding="utf-8") as file:
    for sentence in sentences:
        file.write(sentence + "\n")

In [51]:
# Имя файла для записи
filename = "sentences_3.txt"

# Запись предложений в файл
with open(filename, "w", encoding="utf-8") as file:
    for sentence in sentences:
        file.write(sentence + "\n")

In [39]:
# Имя файла для записи
filename = "sentences_2.txt"

# Запись предложений в файл
with open(filename, "w", encoding="utf-8") as file:
    for sentence in sentences:
        file.write(sentence + "\n")

## *Обучение моделей FastText и проверка результатов на примерах*

In [3]:
import fasttext

# Путь к предобученной модели
pretrained_model_path = "marus_data/cc.en.300.bin"

# Загрузка предобученной модели
pretrained_model = fasttext.load_model(pretrained_model_path)

In [1]:
#test
import fasttext

# Путь к предобученной модели
pretrained_model_path = "updated_model_en_3.bin"

# Загрузка предобученной модели
pretrained_model_11 = fasttext.load_model(pretrained_model_path)

In [7]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "Thunderstruck"
word2 = "thanderstark"

# Get word vectors
vec1 = pretrained_model_11.get_word_vector(word1)
vec2 = pretrained_model_11.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'Thunderstruck' and 'thanderstark': 0.9343559145927429


In [8]:
import fasttext

# Путь к предобученной модели
pretrained_model_path = "updated_model_en_4.bin"

# Загрузка предобученной модели
pretrained_model = fasttext.load_model(pretrained_model_path)

In [6]:
from numpy.linalg import norm

In [9]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "Enter the sandman"
word2 = "byutifl sings"

# Get word vectors
vec1 = pretrained_model.get_word_vector(word1)
vec2 = pretrained_model.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'Enter the sandman' and 'byutifl sings': 0.6255127787590027


In [10]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "lovemeallasudden"
word2 = "love me all of a sudden"

# Get word vectors
vec1 = pretrained_model.get_word_vector(word1)
vec2 = pretrained_model.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'lovemeallasudden' and 'love me all of a sudden': 0.9867682456970215


In [11]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "seven rings"
word2 = "7 Rings"

# Get word vectors
vec1 = pretrained_model.get_word_vector(word1)
vec2 = pretrained_model.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'seven rings' and '7 Rings': 0.8639674186706543


In [36]:
from numpy.linalg import norm

In [12]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "mother"
word2 = "mom"

# Get word vectors
vec1 = pretrained_model.get_word_vector(word1)
vec2 = pretrained_model.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'mother' and 'mom': 0.592470645904541


In [40]:
pretrained_model = fasttext.train_unsupervised(
    input='sentences_2.txt',
    epoch=5       # Количество эпох
)

# Сохранение обновленной модели
pretrained_model.save_model("updated_model_en_2.bin")

Read 0M words
Number of words:  4463
Number of labels: 0
Progress: 100.0% words/sec/thread:   89811 lr:  0.000000 avg.loss:  2.814591 ETA:   0h 0m 0s


In [44]:
pretrained_model = fasttext.train_unsupervised(
    input='sentences_2.txt',
    epoch=10       # Количество эпох
)

# Сохранение обновленной модели
pretrained_model.save_model("updated_model_en_3.bin")

Read 0M words
Number of words:  4463
Number of labels: 0
Progress: 100.0% words/sec/thread:   90288 lr:  0.000000 avg.loss:  2.270929 ETA:   0h 0m 0s


In [54]:
pretrained_model = fasttext.train_unsupervised(
    input='sentences_3.txt',
    epoch=5       # Количество эпох
)

# Сохранение обновленной модели
pretrained_model.save_model("updated_model_en_4.bin")

Read 8M words
Number of words:  70031
Number of labels: 0
Progress: 100.0% words/sec/thread:   40407 lr:  0.000000 avg.loss:  1.248244 ETA:   0h 0m 0s


In [7]:
pretrained_model = fasttext.train_unsupervised(
    input='sentences_4.txt',
    epoch=7       # Количество эпох
)

# Сохранение обновленной модели
pretrained_model.save_model("updated_model_en_4.bin")

Read 0M words
Number of words:  6360
Number of labels: 0
Progress: 100.0% words/sec/thread:   80667 lr:  0.000000 avg.loss:  2.223634 ETA:   0h 0m 0s


## *Дополнительные примеры на реальных песнях для разных моеделей*

In [13]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "shape of you"
word2 = "shap ov u"

# Get word vectors
vec1 = pretrained_model.get_word_vector(word1)
vec2 = pretrained_model.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'shape of you' and 'shap ov u': 0.9586991667747498


In [14]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Words to compare
word1 = "shape of you"
word2 = "shap ov u"

# Get word vectors
vec1 = pretrained_model_11.get_word_vector(word1)
vec2 = pretrained_model_11.get_word_vector(word2)

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)

print(f"Similarity between '{word1}' and '{word2}': {similarity}")

Similarity between 'shape of you' and 'shap ov u': 0.980288565158844
